In [51]:
# import libraries
import torchvision
import torch
import cv2
import numpy as np
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.nn as nn

print(device)
# prepare dataset
trainDataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

testDataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

# Confirm they are in grayscale (they should already be).
# should be [1, 28, 28], 1 channel, 28x28 pixels (grayscale)
print(trainDataset[0][0].shape)
print(testDataset[0][0].shape)
def siftFeatureDetector(image):
    sift = cv2.SIFT_create()
    image = (image * 255).astype(np.uint8)
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors
def normalizeImage(image):
    image = image.view(-1, 28 * 28)  
    return (image - 0.5) / 0.5  
def extraDescriptorsAndLabels(dataSet, batch_size=128):
    train_loader = torch.utils.data.DataLoader(
        dataSet, batch_size=batch_size, shuffle=False)

    all_descriptors = []
    all_labels = []

    # use tqdm to show progress bar
    for batch in tqdm(train_loader, desc="Processing Batches", unit="batch"):
        images, labels = batch
        images = images.squeeze().numpy()
        for image in images:
            keypoints, descriptors = siftFeatureDetector(image)
            all_descriptors.append(descriptors)
            all_labels.append(labels)
    # verify that the descriptors and the labels are being stored correctly
    if len(all_descriptors) == len(all_labels) and len(all_descriptors) == len(dataSet):
        return all_descriptors, all_labels
    else:
        print("Error: descriptors and labels are not the same length")
        return None
def siftFeatureDetector(image):
    sift = cv2.SIFT_create()
    image = (image * 255).astype(np.uint8)  
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors if descriptors is not None else np.array([])

def extraDescriptorsAndLabels(dataSet, batch_size=128):
    train_loader = torch.utils.data.DataLoader(
        dataSet, batch_size=batch_size, shuffle=False)

    all_descriptors = []
    all_labels = []

    # use tqdm to show progress bar
    for batch in tqdm(train_loader, desc="Processing Batches", unit="batch"):
        images, labels = batch
        images = images.squeeze().numpy()
        for image in images:
            keypoints, descriptors = siftFeatureDetector(image)
            all_descriptors.append(descriptors)
            all_labels.append(labels)
    # verify that the descriptors and the labels are being stored correctly
    if len(all_descriptors) == len(all_labels) and len(all_descriptors) == len(dataSet):
        return all_descriptors, all_labels
    else:
        print("Error: descriptors and labels are not the same length")
        return None
# get descriptors and labels
train_descriptors, train_labels = extraDescriptorsAndLabels(trainDataset)
class Task1_MLP(nn.Module):
    def __init__(self, input_dim=28 * 28, hidden_dim1=256, hidden_dim2=128, output_dim=10):
        super(Task1_MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)  
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)  
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim2, output_dim)  
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = normalizeImage(x)  
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return self.softmax(x)
def to_one_hot(labels, num_classes=10):
    return torch.nn.functional.one_hot(labels, num_classes=num_classes).float()
#class RMSELoss(nn.Module):
    #def __init__(self):
        #super(RMSELoss, self).__init__()

    #def forward(self, y_pred, y_true):
        #return torch.sqrt(torch.mean((y_pred - y_true) ** 2))  

class RAELoss(nn.Module):
    def __init__(self, eps=1e-8):
        super(RAELoss, self).__init__()
        self.eps = eps  

    def forward(self, y_pred, y_true):
        abs_error = torch.abs(y_pred - y_true)  
        mean_abs_error = torch.mean(abs_error)  
        denominator = torch.mean(torch.abs(y_true - torch.mean(y_true))) + self.eps  
        return mean_abs_error / denominator  

# convert 28*28 image to 784-dim vector
def imageToVector(image):
    return image.view(-1, 28 * 28)


# train the model
model = Task1_MLP().to(device)

# define loss function and optimizer
#criterion = RMSELoss()
criterion = RAELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# train the model
num_epochs = 10
batch_size = 16
train_loader = torch.utils.data.DataLoader(
    trainDataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    loop = tqdm(enumerate(train_loader), total=len(
        train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")

    for i, (images, labels) in loop:
        images = images.to(device)
        labels = to_one_hot(labels)

        # forward pass
        outputs = model(imageToVector(images))
        loss = criterion(outputs, labels)

        # backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) == loop.total:
            print(
                f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item()}')

cpu
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])


Epoch 1/10: 100%|██████████| 3750/3750 [01:52<00:00, 33.40it/s]


Epoch [1/10], Step [3750/3750], Loss: 0.9027777314186096


Epoch 2/10: 100%|██████████| 3750/3750 [02:09<00:00, 28.85it/s]


Epoch [2/10], Step [3750/3750], Loss: 0.972222089767456


...

Epoch [8/10], Step [3750/3750], Loss: 1.111111044883728


Epoch 9/10: 100%|██████████| 3750/3750 [02:10<00:00, 28.78it/s]


Epoch [9/10], Step [3750/3750], Loss: 0.972222089767456


Epoch 10/10: 100%|██████████| 3750/3750 [02:10<00:00, 28.77it/s]

Epoch [10/10], Step [3750/3750], Loss: 1.0416665077209473


In [52]:
def evaluate_model(model, data_loader, dataset_name="Test Set"):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(imageToVector(images))
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"{dataset_name} Accuracy: {accuracy:.2f}%")
    return accuracy

In [53]:
test_loader = torch.utils.data.DataLoader(testDataset, batch_size=batch_size, shuffle=False)
evaluate_model(model, train_loader, "Training Set")
evaluate_model(model, test_loader, "Test Set")

Training Set Accuracy: 10.00%
Test Set Accuracy: 10.00%


10.0